In [1]:
import numpy as np
import tensorflow as tf
import os
import pandas as pd

c:\program files (x86)\python\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\program files (x86)\python\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\program files (x86)\python\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\program files (x86)\python\lib\site-packages\tensorflow\python\framework\dtypes.py:519: 

# CHECK MODEL ACCURACY
Bagian dibawah ini merupakan kode untuk memeriksa akurasi model

In [2]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16,input_dim=13, kernel_initializer='normal', activation='relu'),
  tf.keras.layers.Dense(32,kernel_initializer='normal', activation='relu'),
  tf.keras.layers.Dense(32,kernel_initializer='normal', activation='relu'),
  tf.keras.layers.Dense(8,kernel_initializer='normal', activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])
adam = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=adam,
                loss='binary_crossentropy',
                metrics=['acc'])
print(model.summary())

W0619 05:14:34.508421  5404 deprecation.py:506] From c:\program files (x86)\python\lib\site-packages\tensorflow\python\keras\initializers.py:143: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0619 05:14:34.508421  5404 deprecation.py:506] From c:\program files (x86)\python\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0619 05:14:34.617788  5404 deprecation.py:323] From c:\program files (x86)\python\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.a

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                224       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 9         
Total params: 2,097
Trainable params: 2,097
Non-trainable params: 0
_________________________________________________________________
None


In [3]:
checkpoint_filepath = './best_model/ckpt{epoch}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_filepath)
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
model.load_weights(latest)

./best_model\ckpt_52


In [4]:
# preprocess the data (don't put this block in build app)
dataset = pd.read_csv('heart.csv', sep=';')
data = dataset[~dataset.isin(['?'])]
data = data.dropna(axis=0)
# Ubah target 2,3,4 jadi 1
data.loc[data['target'] > 1, 'target'] = 1
data = data.apply(pd.to_numeric)
X = np.array(data.drop(['target'], 1))
y = np.array(data['target'])

# Re-evaluate the model unutk semua dataset
loss,acc = model.evaluate(X,  y, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

297/297 - 1s - loss: 0.3746 - acc: 0.8586
Restored model, accuracy: 85.86%


In [5]:
predictY = model.predict(X)
hasilPredict = (predictY >= 0.5).astype(int)
from sklearn.metrics import classification_report,roc_auc_score,roc_curve
print(classification_report(y,hasilPredict))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87       160
           1       0.87      0.82      0.84       137

    accuracy                           0.86       297
   macro avg       0.86      0.86      0.86       297
weighted avg       0.86      0.86      0.86       297



# ALUR PREPROCESSING DATASET
Bagian dibawah ini merupakan kode preprocessing dataset

In [21]:
dataset = pd.read_csv('heart.csv', sep=';')
print('Jumlah data: '+ str(dataset.shape))
dataset.loc[295:]

Jumlah data: (303, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
295,41.0,1.0,2.0,120.0,157.0,0.0,0.0,182.0,0.0,0.0,1.0,0.0,3.0,0
296,59.0,1.0,4.0,164.0,176.0,1.0,2.0,90.0,0.0,1.0,2.0,2.0,6.0,3
297,57.0,0.0,4.0,140.0,241.0,0.0,0.0,123.0,1.0,0.2,2.0,0.0,7.0,1
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1
302,38.0,1.0,3.0,138.0,175.0,0.0,0.0,173.0,0.0,0.0,1.0,?,3.0,0


In [24]:
# remove missing data (indicated with a "?")
data = dataset[~dataset.isin(['?'])]
# drop rows with NaN values from DataFrame
data = data.dropna(axis=0)
print('Jumlah data: '+ str(data.shape))
data.loc[295:]

Jumlah data: (297, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
295,41.0,1.0,2.0,120.0,157.0,0.0,0.0,182.0,0.0,0.0,1.0,0.0,3.0,0
296,59.0,1.0,4.0,164.0,176.0,1.0,2.0,90.0,0.0,1.0,2.0,2.0,6.0,3
297,57.0,0.0,4.0,140.0,241.0,0.0,0.0,123.0,1.0,0.2,2.0,0.0,7.0,1
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1


In [25]:
# Ubah target 2,3,4 jadi 1
data.loc[data['target'] > 1, 'target'] = 1
print('Jumlah data: '+ str(data.shape))
data.loc[295:]

Jumlah data: (297, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
295,41.0,1.0,2.0,120.0,157.0,0.0,0.0,182.0,0.0,0.0,1.0,0.0,3.0,0
296,59.0,1.0,4.0,164.0,176.0,1.0,2.0,90.0,0.0,1.0,2.0,2.0,6.0,1
297,57.0,0.0,4.0,140.0,241.0,0.0,0.0,123.0,1.0,0.2,2.0,0.0,7.0,1
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,1
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,1
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1
